In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader


In [9]:
class SolarDataset(Dataset):
    def __init__(self, data_array, targets_array=None):
        self.scaler = StandardScaler()
        self.data = self.scaler.fit_transform(data_array)

        if targets_array is not None:
            self.targets = targets_array
        else:
            self.targets = None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_sample = torch.tensor(self.data[idx], dtype=torch.float32)

        if self.targets is not None:
            target_sample = torch.tensor(self.targets[idx], dtype=torch.float32)
            return data_sample, target_sample
        else:
            return data_sample


In [5]:
class SolarNet(nn.Module):
    def __init__(self, input_dim):
        super(SolarNet, self).__init__()

        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.fc5 = nn.Linear(128, 1)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = F.leaky_relu(self.bn4(self.fc4(x)))
        x = self.dropout(x)
        return self.fc5(x)
